We faced computation limits issues so we used pythin for phase 2 computations

This two-phase approach reduces memory usage to avoid exceeding GEE's user memory limits.


In [1]:
# Cell 1: Import Libraries and Initialize GEE
# ---------------------------------------------
import ee
import geemap
import pandas as pd
import numpy as np
import time

# Authenticate and initialize GEE with project ID
try:
    project_id = 'ee-testing-casa-25'
    ee.Initialize(project=project_id)
    print("GEE initialized successfully.")
except ee.ee_exception.EEException as e:
    print("Authentication or initialization failed:", str(e))
    print("Attempting to authenticate...")
    ee.Authenticate()
    ee.Initialize(project=project_id)

# Test access to confirm initialization
try:
    test_asset = ee.FeatureCollection("projects/ee-testing-casa-25/assets/Nepal_boundary")
    print("Number of features in Nepal_boundary:", test_asset.size().getInfo())
except Exception as e:
    print("Failed to access test asset:", str(e))
    raise

GEE initialized successfully.
Number of features in Nepal_boundary: 1


In [2]:
# Cell 2: Load Data Assets and Inspect Band Names
# ---------------------------------------------
# Load the intermediate FeatureCollection from Phase 1
districtFactors = ee.FeatureCollection("projects/ee-testing-casa-25/assets/NepalDistrictFactors_Intermediate")

# Load the Nepal boundary for clipping rasters
NepalBoundary = ee.FeatureCollection("projects/ee-testing-casa-25/assets/Nepal_boundary")

# Load the susceptibility rasters
LandslideSusceptibility = ee.Image("projects/ee-clareluikart/assets/landslide_sus").clip(NepalBoundary)
LandslideSusceptibilityPopOnly = ee.Image("projects/ee-testing-casa-25/assets/Normalized_Landslide_Risk_Pop_only").clip(NepalBoundary)
RFProbabilityPopOnly = ee.Image("projects/ee-testing-casa-25/assets/RF_Landslide_Propbability_Pop_only").clip(NepalBoundary)

# Diagnostic: Inspect band names of all images
print("LandslideSusceptibility band names:", LandslideSusceptibility.bandNames().getInfo())
print("LandslideSusceptibilityPopOnly band names:", LandslideSusceptibilityPopOnly.bandNames().getInfo())
print("RFProbabilityPopOnly band names:", RFProbabilityPopOnly.bandNames().getInfo())


LandslideSusceptibility band names: ['slope']
LandslideSusceptibilityPopOnly band names: ['risk']
RFProbabilityPopOnly band names: ['classification']


In [3]:
# Cell 3: Process Districts and Compute Metrics
# ---------------------------------------------
# Convert FeatureCollection to a list of features for processing in Python
features = districtFactors.getInfo()['features']
districts_data = []

# Process each district
for idx, feature in enumerate(features):
    district_name = feature['properties']['DISTRICT']
    district_geom = ee.Geometry(feature['geometry']).simplify(100)

    # Batch reduceRegions operations for LandslideSusceptibility
    combined_reducer = ee.Reducer.mean().combine(
        reducer2=ee.Reducer.stdDev(),
        sharedInputs=True
    ).combine(
        reducer2=ee.Reducer.sum(),
        sharedInputs=True
    )
    stats = LandslideSusceptibility.reduceRegion(
        reducer=combined_reducer,
        geometry=district_geom,
        scale=100,
        maxPixels=1e10,
        bestEffort=True
    ).getInfo()
    
    # Fix avgSusceptibility computation: use 'slope_mean' key
    avgSusceptibility = max(float(stats.get('slope_mean', 0)), 0)
    susceptibilityStdDev = max(float(stats.get('slope_stdDev', 0)), 0)
    
    # Diagnostic: Print stats for the first few districts
    if idx < 5:
        print(f"\nDistrict: {district_name}")
        print(f"Stats: {stats}")
        print(f"Avg Susceptibility: {avgSusceptibility}")
        print(f"StdDev Susceptibility: {susceptibilityStdDev}")
        
        pixel_count = LandslideSusceptibility.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=district_geom,
            scale=100,
            maxPixels=1e10,
            bestEffort=True
        ).get('slope').getInfo()
        print(f"Number of pixels in district: {pixel_count}")

    # Compute high-risk area (susceptibility > 0.5)
    highRiskMask = LandslideSusceptibility.gt(0.5).selfMask()
    
    # Diagnostic: Check the band names of highRiskMask
    highRiskMask_bands = highRiskMask.bandNames().getInfo()
    print(f"District {district_name} - highRiskMask band names: {highRiskMask_bands}")
    
    # Use the correct band name (likely 'slope', not 'constant')
    highRiskArea_dict = highRiskMask.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=district_geom,
        scale=100,
        maxPixels=1e10,
        bestEffort=True
    ).getInfo()
    
    # Handle empty results: default to 0 if no pixels are unmasked
    highRiskArea = highRiskArea_dict.get('slope', 0)  # Use 'slope' or adjust based on diagnostic output
    totalArea = float(feature['properties']['area_km2'])
    highRiskPercentage = max((highRiskArea or 0) / totalArea * 100, 0)

    # Compute average susceptibility for populated areas only
    avgSusceptibilityPopOnly = LandslideSusceptibilityPopOnly.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=district_geom,
        scale=100,
        maxPixels=1e10,
        bestEffort=True
    ).get('risk').getInfo()  # Updated to use 'risk' band
    avgSusceptibilityPopOnly = max(float(avgSusceptibilityPopOnly or 0), 0)

    # Compute average Random Forest probability for populated areas only
    avgRFProbabilityPopOnly = RFProbabilityPopOnly.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=district_geom,
        scale=100,
        maxPixels=1e10,
        bestEffort=True
    ).get('classification').getInfo()
    avgRFProbabilityPopOnly = max(float(avgRFProbabilityPopOnly or 0), 0)

    # Update feature properties
    data = feature['properties'].copy()
    data['geometry'] = feature['geometry']
    data.update({
        'avgSusceptibility': avgSusceptibility,
        'highRiskPercentage': highRiskPercentage,
        'susceptibilityStdDev': susceptibilityStdDev,
        'avgSusceptibilityPopOnly': avgSusceptibilityPopOnly,
        'avgRFProbabilityPopOnly': avgRFProbabilityPopOnly
    })
    districts_data.append(data)

    print(f"Processed district {idx + 1}/{len(features)}: {district_name}")


District: MAHOTTARI
Stats: {'slope_mean': 0.37394924163390975, 'slope_stdDev': 0.06892085904035883, 'slope_sum': 40372.07045752779}
Avg Susceptibility: 0.37394924163390975
StdDev Susceptibility: 0.06892085904035883
Number of pixels in district: 110690
District MAHOTTARI - highRiskMask band names: ['slope']
Processed district 1/77: MAHOTTARI

District: SIRAHA
Stats: {'slope_mean': 0.36306664256855575, 'slope_stdDev': 0.06400594500336383, 'slope_sum': 44368.4124046403}
Avg Susceptibility: 0.36306664256855575
StdDev Susceptibility: 0.06400594500336383
Number of pixels in district: 125097
District SIRAHA - highRiskMask band names: ['slope']
Processed district 2/77: SIRAHA

District: BANKE
Stats: {'slope_mean': 0.3175227079002539, 'slope_stdDev': 0.08253950042145433, 'slope_sum': 65704.6402356593}
Avg Susceptibility: 0.3175227079002539
StdDev Susceptibility: 0.08253950042145433
Number of pixels in district: 211175
District BANKE - highRiskMask band names: ['slope']
Processed district 3/77:

In [4]:
# Cell 4: Create DataFrame and Compute Additional Metrics
# ---------------------------------------------
# Create a DataFrame
df = pd.DataFrame(districts_data)

# Compute national average susceptibility for relativeRiskIndex
national_avg_susceptibility = df['avgSusceptibility'].mean()
df['relativeRiskIndex'] = df['avgSusceptibility'].apply(lambda x: x / national_avg_susceptibility if national_avg_susceptibility > 0 else 0)

# Compute incident density metrics
df['incidentsPerKm2'] = df['incidentCount'] / df['area_km2'].replace(0, 0.001)
df['deathsPerKm2'] = df['deaths'] / df['area_km2'].replace(0, 0.001)
df['injuriesPerKm2'] = df['injuries'] / df['area_km2'].replace(0, 0.001)
df['infraDestroyedPerKm2'] = df['infraDestroyed'] / df['area_km2'].replace(0, 0.001)

# Compute incident percentage
total_incidents = df['incidentCount'].sum()
df['incidentPercentage'] = df['incidentCount'].apply(lambda x: (x / total_incidents * 100) if total_incidents > 0 else 0)

# Compute centroids for nearest neighbor calculations
df['centroid_lon'] = df.apply(lambda row: ee.Geometry(row['geometry']).centroid().coordinates().get(0).getInfo(), axis=1)
df['centroid_lat'] = df.apply(lambda row: ee.Geometry(row['geometry']).centroid().coordinates().get(1).getInfo(), axis=1)

# Compute distances and find nearest neighbors
neighbors = []
for idx, row in df.iterrows():
    district_name = row['DISTRICT']
    lon1, lat1 = row['centroid_lon'], row['centroid_lat']
    
    distances = []
    for idx2, row2 in df.iterrows():
        if row2['DISTRICT'] == district_name:
            continue
        lon2, lat2 = row2['centroid_lon'], row2['centroid_lat']
        distance = ((lon1 - lon2) ** 2 + (lat1 - lat2) ** 2) ** 0.5
        distances.append((row2['DISTRICT'], distance))
    
    distances.sort(key=lambda x: x[1])
    neighbor1, neighbor2 = distances[0][0], distances[1][0]
    neighbors.append((neighbor1, neighbor2))

# Add neighbor names to DataFrame
df['neighbor1'], df['neighbor2'] = zip(*neighbors)

# Drop temporary and unnecessary columns
df = df.drop(columns=['centroid_lon', 'centroid_lat', 'geometry'])

# Export to CSV
df.to_csv('NepalDistrictCalculatedFactors_V4.csv', index=False)
print("Exported to NepalDistrictCalculatedFactors_V4.csv")


Exported to NepalDistrictCalculatedFactors_V4.csv


In [5]:
# Cell 5: Inspect Results
# ---------------------------------------------
print("DataFrame Head:")
print(df[['DISTRICT', 'avgSusceptibility', 'susceptibilityStdDev', 'avgSusceptibilityPopOnly', 'avgRFProbabilityPopOnly']].head())

DataFrame Head:
           DISTRICT  avgSusceptibility  susceptibilityStdDev  \
0         MAHOTTARI           0.373949              0.068921   
1            SIRAHA           0.363067              0.064006   
2             BANKE           0.317523              0.082540   
3  NAWALPARASI WEST           0.338470              0.073805   
4              BARA           0.344370              0.067537   

   avgSusceptibilityPopOnly  avgRFProbabilityPopOnly  
0                  0.440293                 0.090692  
1                  0.430409                 0.096641  
2                  0.416215                 0.080888  
3                  0.419662                 0.122425  
4                  0.430015                 0.091445  
